In [8]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken
from conllu import parse_incr, parse


# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


In [3]:
dir = Path("~/Datasets/SRBendding").expanduser()
dir.mkdir(parents=True, exist_ok=True)
dir

PosixPath('/home/selena/Datasets/SRBendding')

In [10]:
file_path = "datasets/train_all.conllup"

data_file = open(file_path, "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    print(tokenlist)

TokenList<id=1, id=2, id=3, id=4, id=5, id=6, id=7, id=8, id=9, id=10, id=11, id=12, id=13, id=14, id=15, id=16, id=17, id=18, id=19, id=20, id=21, id=22, metadata={global.columns: "ID TOKEN NORM LEMMA UPOS XPOS FEATS NER_TYPE", sent_id: "tid.490437095392301056.1"}>
TokenList<id=1, id=2, id=3, metadata={sent_id: "tid.490437095392301056.2"}>
TokenList<id=1, id=2, id=3, id=4, id=5, id=6, id=7, id=8, id=9, id=10, id=11, id=12, id=13, id=14, id=15, id=16, id=17, id=18, id=19, id=20, id=21, id=22, id=23, id=24, metadata={sent_id: "tid.506074507464179713.1"}>
TokenList<id=1, id=2, id=3, id=4, id=5, id=6, id=7, id=8, id=9, id=10, id=11, id=12, id=13, id=14, id=15, id=16, id=17, id=18, id=19, id=20, id=21, id=22, id=23, id=24, id=25, metadata={sent_id: "tid.386277981196283904.1"}>
TokenList<id=1, id=2, id=3, id=4, id=5, id=6, id=7, id=8, id=9, id=10, id=11, id=12, id=13, id=14, id=15, id=16, id=17, id=18, id=19, id=20, id=21, id=22, id=23, id=24, id=25, id=26, id=27, id=28, id=29, id=30, id=31

In [24]:
# sentences = parse(tokenlist)

In [23]:
# from conllu import parse_incr

# file_path = "datasets/train_all.conllup"

# with open(file_path, "r", encoding="utf-8") as data_file:
#     for tokenlist in parse_incr(data_file):
#         # Extract the words to form a sentence
#         sentence = " ".join(token['form'] for token in tokenlist)
        
#         # Extract sentence ID from metadata if available
#         sentence_id = tokenlist.metadata.get('sent_id', 'N/A')
        
#         # Print the sentence and its ID
#         print(f"Sentence ID: {sentence_id}")
#         print(f"Sentence: {sentence}\n")


In [19]:

# Path to your .conllup file
file_path = "datasets/train_all.conllup"
sentences = []
ids = []

# Open the file and parse it incrementally
with open(file_path, "r", encoding="utf-8") as data_file:
    for tokenlist in parse_incr(data_file):
        # Extract the words (tokens) to form a sentence
        sentence = " ".join(token['token'] for token in tokenlist)
        sentences.append(sentence)
        # Extract sentence ID from metadata if available
        sentence_id = tokenlist.metadata.get('sent_id', 'N/A')
        ids.append(sentence_id)
        # Print the sentence and its ID
        # print(f"Sentence ID: {sentence_id}")
        # print(f"Sentence: {sentence}\n")


In [20]:
len(sentence), len(sentence_id), len(set(sentence_id))

(116, 24, 14)

In [22]:
sentences

['@vukomand Gospođo Dijana koje lekove vaš šef koristi i da li se isti mogu prepisati na recept ili se moraju kupiti ??',
 'Unapred hvala .',
 'Moja verenica je prepoznala Soraju iz Gruovog spota Ti To Zoveš ljubav po sisama , kaže tako joj liči , a glava ništa .',
 'Žao mi je gospa Lejdi Gago al mi smo sve to već videli kod Slađane Milošević pre mnogo godina al sam mnogo više ukusa ...',
 'Kada shvatiš da ti ona ne znači više , tada u tamnoj noći sunce ti svane , ali ako su emocije kao kapi kiše , kišobran dižem kada kiša stane .',
 'Prvo su je pravili kao Skadar na Bojani ..',
 'Sad je prepravljaju kao da je ikad i valjala .',
 'E Vlado Srpska u cije si ruke upala !',
 'Blago nama i vama !',
 'Ah , koliko stvari mogu da uradim , stvorim , izgradim , darujem , podelim , sastavim i popravim ...',
 'Mogu mnogo ali ne mogu sam .',
 '@MDamnjanovic Hvala !',
 'Šteta što im sajt nije kompletan .',
 'Ako se ispostavi da " Zorba " ne radi , ovo bi moglo biti zanimljivo .',
 'Prema snegu , kao